In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('recommender').getOrCreate()
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator

In [2]:
df = spark.read.csv('/FileStore/tables/ratings_full.csv', inferSchema=True, header=True)

In [3]:
df.printSchema()

In [4]:
df.columns

In [5]:
df = df.select(['userId', 'movieId', 'rating'])

In [6]:
df.head(5)

In [7]:
df.show()


In [8]:
df.describe().show()

In [9]:
training, test = df.randomSplit([0.8,0.2])

In [10]:
als = ALS(maxIter=5, regParam=0.01, userCol='userId', itemCol='movieId', ratingCol='rating')

model = als.fit(training)

predictions = model.transform(test)

In [11]:
predictions.describe().show()

In [12]:
predictions = predictions.na.drop()


In [13]:
predictions.describe().show()


In [14]:
evaluator = RegressionEvaluator(metricName='rmse', labelCol='rating')
rmse = evaluator.evaluate(predictions)
rmse